In [1]:
%config IPCompleter.greedy=True

In [2]:
from influxdb import InfluxDBClient
import getpass
import pandas as pd

In [3]:
dataLocation = "../data/"
!export dataLocation="../data/"

In [5]:
name = input()
password = getpass.getpass()

nimac
········


In [6]:
client = InfluxDBClient('localhost', 8086, name, password, 'SG')

In [7]:
def makeJson(time, temp, humid, M1, M2, M3):
    return {
            "measurement": "garden",
            "tags": {
                "host": "server01",
                "region": "us-west"
            },
    #         "time": "2009-11-10T23:00:00Z",
            "time": str(time),
            "fields": {
                "temp": temp,
                "humid": humid,
                "M1": M1,
                "M2": M2,
                "M3": M3
            }
        }

In [11]:
def readCsv(dataLocation, csvFileName):
    df=pd.read_csv(dataLocation + csvFileName + '.csv', sep=',',header=[0])
    return df

df = readCsv(dataLocation, "master")
df["time"] = pd.to_datetime(df['time'], unit='ms')
json_body = []

done = 0

for i in range(len(df)):
    time, temp, humid, M1, M2, M3 = df.iloc[i]
    json_body.append(makeJson(time, temp, humid, M1, M2, M3))
    if((i * 100) // len(df) > done):
        print(done+1)
        done = (i*100)//len(df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


In [9]:
client.write_points(json_body)

True

In [ ]:
result = client.query('select temp from garden;')
print("Result: {0}".format(result))